<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import d2l
import mxnet
from mxnet import gluon,npx,np
from mxnet.gluon import nn
npx.set_np()

<p>To understand how resnet work read </p>
<a href='https://arxiv.org/pdf/1512.03385.pdf'>Deep Residual Learning for Image Recognition<a/>
    
<a href='https://arxiv.org/pdf/1603.05027.pdf'>Identity Mappings in Deep Residual Networks</a>

<img src="images/resnet.jpg"  width='1000px'>
Source:  <a href='https://arxiv.org/pdf/1512.03385.pdf'>Deep Residual Learning for Image Recognition<a/>
  

In [2]:
class Residual(nn.Block):
    def __init__(self,num_channels,strides=1,downsample=False,**kwargs):
        super().__init__(**kwargs)
        self.cov1=nn.Conv2D(num_channels,kernel_size=3,strides=strides,padding=1)
        self.cov2=nn.Conv2D(num_channels,kernel_size=3,padding=1)
        if downsample:
            self.downsample=nn.Conv2D(num_channels,kernel_size=1,strides=strides)
        else:
            self.downsample=None
        self.bn1=nn.BatchNorm()
        self.bn2=nn.BatchNorm()
        
    def forward(self,x):
        h_x=npx.relu(self.bn1(self.cov1(x)))
        h_x=self.bn2(self.cov2(h_x))
        if self.downsample:
            x=self.downsample(x)
        return npx.relu(h_x+x)

In [3]:
def resnet_block(num_channels,num_residuals,first_block=False):
    blk=nn.Sequential()
    for i in range(num_residuals):
        if i==0 and not first_block:
            blk.add(Residual(num_channels,downsample=True,strides=2))
        else:
            blk.add(Residual(num_channels))
    return blk

resnet34=nn.Sequential()
resnet34.add(  nn.Conv2D(64,kernel_size=7,strides=2,padding=3),
             nn.BatchNorm(),
             nn.Activation('relu'),
             nn.MaxPool2D(pool_size=3,strides=2,padding=1),
             resnet_block(num_channels=64,num_residuals=3,first_block=True),
             resnet_block(128,4),
             resnet_block(256,6),
             resnet_block(512,3),
             nn.GlobalAvgPool2D(),
             nn.Dense(10))

In [4]:
X=np.random.uniform(size=(1,1,28,28))
resnet34.initialize()

In [5]:
for layer in resnet34:
    X=layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv0 output shape:	 (1, 64, 14, 14)
batchnorm0 output shape:	 (1, 64, 14, 14)
relu0 output shape:	 (1, 64, 14, 14)
pool0 output shape:	 (1, 64, 7, 7)
sequential1 output shape:	 (1, 64, 7, 7)
sequential2 output shape:	 (1, 128, 4, 4)
sequential3 output shape:	 (1, 256, 2, 2)
sequential4 output shape:	 (1, 512, 1, 1)
pool1 output shape:	 (1, 512, 1, 1)
dense0 output shape:	 (1, 10)
